<img width="100" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Forest Emissions Tracking - Validation

_CarbonPlan ClimateTrace Team_

This notebook compares our estimates of country-level forest emissions to prior estimates from other
groups. The notebook currently compares againsts:

- Global Forest Watch (Zarin et al. 2016)
- Global Carbon Project (Friedlingstein et al. 2020)


In [ ]:
import geopandas
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import xarray as xr
from carbonplan_styles.mpl import set_theme
import urllib3

urllib3.disable_warnings()
set_theme()

In [ ]:
# Input data
# ----------

# country shapes from GADM36
countries = geopandas.read_file("s3://carbonplan-climatetrace/inputs/shapes/countries.shp")

# CarbonPlan's emissions
emissions = pd.read_csv("s3://carbonplan-climatetrace/v0.4/country_rollups_emissions.csv")

# GFW emissions
gfw_emissions = pd.read_excel(
    "s3://carbonplan-climatetrace/validation/gfw_global_emissions.xlsx",
    sheet_name="Country co2 emissions",
).dropna(axis=0)
gfw_emissions = gfw_emissions[gfw_emissions["threshold"] == 10]  # select threshold

# Global Carbon Project
gcp_emissions = (
    pd.read_excel(
        "s3://carbonplan-climatetrace/validation/Global_Carbon_Budget_2020v1.0.xlsx",
        sheet_name="Land-Use Change Emissions",
        skiprows=28,
    )
    .dropna(axis=1)
    .set_index("Year")
)
gcp_emissions *= 3.664  # C->CO2
gcp_emissions.index = [pd.to_datetime(f"{y}-01-01") for y in gcp_emissions.index]
gcp_emissions = gcp_emissions[["GCB", "H&N", "BLUE", "OSCAR"]]

In [ ]:
v1_emissions_fire = pd.read_csv(
    "s3://carbonplan-climatetrace/v1.2/country_rollups_emissions_from_fire.csv"
)
v1_emissions_clearing = pd.read_csv(
    "s3://carbonplan-climatetrace/v1.2/country_rollups_emissions_from_clearing.csv"
)
v1_sinks = pd.read_csv("s3://carbonplan-climatetrace/v1.2/country_rollups_sinks.csv")
v1_net = pd.read_csv("s3://carbonplan-climatetrace/v1.2/country_rollups_net.csv")

In [ ]:
v1_total = v1_emissions_fire.copy()
v1_total["tCO2eq"] = v1_emissions_fire["tCO2eq"] + v1_emissions_clearing["tCO2eq"]

In [ ]:
# Merge emissions dataframes with countries GeoDataFrame
gfw_counties = countries.merge(gfw_emissions.rename(columns={"country": "name"}), on="name")
trace_counties = countries.merge(emissions.rename(columns={"iso3_country": "alpha3"}), on="alpha3")

In [ ]:
# reformat to "wide" format (time x country)
emissions_dfs, trace_wide = {}, {}
emissions_dfs["v1"] = v1_total
emissions_dfs["v0"] = emissions
emissions_dfs["v1-net"] = v1_net
emissions_dfs["v1-sinks"] = v1_sinks

In [ ]:
for version, df in emissions_dfs.items():

    trace_wide[version] = (
        df.drop(columns=["end_date"])
        .pivot(index="begin_date", columns="iso3_country")
        .droplevel(0, axis=1)
    )
    trace_wide[version].index = pd.to_datetime(trace_wide[version].index)
    trace_wide[version] = trace_wide[version].truncate(before="2015")
gfw_wide = gfw_emissions.set_index("country").filter(regex="whrc_aboveground_co2_emissions_Mg_.*").T
gfw_wide.index = [pd.to_datetime(f"{l[-4:]}-01-01") for l in gfw_wide.index]

gfw_wide.head()

## Part 1 - Compare time-averaged country emissions (tropics only)


In [ ]:
# Create a new dataframe with average emissions
avg_emissions = countries.set_index("alpha3")
avg_emissions["trace-v0"] = trace_wide["v0"].mean().transpose() / 1e9
avg_emissions["trace-v1"] = trace_wide["v1"].mean().transpose() / 1e9
avg_emissions["trace-v1-net"] = trace_wide["v1-net"].mean().transpose() / 1e3
avg_emissions["trace-v1-sinks"] = trace_wide["v1-sinks"].mean().transpose() / 1e9


avg_emissions = avg_emissions.set_index("name")
avg_emissions["gfw"] = gfw_wide.mean().transpose() / 1e9

In [ ]:
# Scatter Plot

avg_emissions.plot.scatter("trace-v1", "trace-v0")
plt.ylabel("Trace-v1 [Tg CO2e]")
plt.xlabel("Trace-v0 [Tg CO2e]")

plt.plot([0, 1.2], [0, 1.2], "k")
plt.ylim(0, 1.2)
plt.xlim(0, 1.2)

## Part 2 - Maps of Tropical Emissions


In [ ]:
avg_emissions_nonan = avg_emissions.dropna()

In [ ]:
avg_emissions

In [ ]:
kwargs = dict(
    legend=True,
    legend_kwds={"orientation": "horizontal", "label": "Emissions [Tg CO2e]"},
    lw=0.25,
    cmap="fire_light",
    vmin=0,
    vmax=1,
)
avg_emissions.plot("trace-v0", **kwargs)
plt.title("Trace-v0 emissions (avg. 2015-2020)")

In [ ]:
avg_emissions.plot("trace-v1", **kwargs)

plt.title("Trace-v1 emissions (avg. 2015-2020)")

In [ ]:
kwargs = dict(
    legend=True,
    legend_kwds={
        "orientation": "horizontal",
        "label": "Emissions Difference [%]",
    },
    lw=0.25,
    cmap="RdBu_r",
    vmin=-1000,
    vmax=100,
)
avg_emissions["pdiff"] = (
    (avg_emissions["trace-v1"] - avg_emissions["trace-v0"]) / avg_emissions["trace-v0"]
) * 100
avg_emissions.plot("pdiff", **kwargs)
plt.title("Difference in emissions 2015-2020 (v1 - v0)")

In [ ]:
kwargs = dict(
    legend=True,
    legend_kwds={"orientation": "horizontal", "label": "Emissions [Tg CO2e]"},
    lw=0.25,
    cmap="water_light_r",
    vmin=-1,
    vmax=0,
)
avg_emissions.plot("trace-v1-sinks", **kwargs)
plt.title("Trace v1")

In [ ]:
kwargs = dict(
    legend=True,
    legend_kwds={
        "orientation": "horizontal",
        "label": "Net emissions [kt CO2]",
    },
    lw=0.25,
    cmap="orangeblue_light_r",
    vmin=-1e5,
    vmax=1e5,
)
avg_emissions.plot("trace-v1-net", **kwargs)

plt.title("Net CO2 flux v1 (average 2015-2020)")

gcp_emissions## Part 3 - Compare global emissions timeseries to Global Carbon Project


In [ ]:
ax = gcp_emissions[["H&N", "BLUE", "OSCAR"]].loc["2000":].plot(ls="--")
gcp_emissions["GCB"].loc["2000":].plot(ax=ax, label="GCB", lw=3)
(trace_wide["v0"].sum(axis=1) / 1e9).plot(ax=ax, label="Trace v0", c="k", lw=3)
(trace_wide["v1"].sum(axis=1) / 1e9).plot(ax=ax, label="Trace v1", c="g", lw=3)
plt.ylabel("Emissions [Tg CO2e]")
plt.legend()